**Processing Pipeline 2**

In [1]:
import os
import polars as pl
import pandas as pd
from datetime import datetime
import chardet

In [160]:
input_df = pl.read_csv("data/PP_ALL.csv", encoding="utf-8")
output_df = pl.read_csv("data/Joined_rtf_files.csv", encoding="utf-8")
input_df = input_df.with_columns(pl.col("Datum").str.strptime(pl.Datetime, "%m/%d/%Y %H:%M"))
output_df = output_df.with_columns(pl.col("Datum").str.strptime(pl.Datetime, "%m/%d/%Y %H:%M"))

In [101]:
from datetime import timedelta
def get_first_inputs_before_time(n, output_df_row, input_df, time_window_after_minutes=60):

    #'04/30/2022 18:30'
    target_datetime = dict(zip(output_df.columns, output_df_row))["Datum"]
    start_time = target_datetime.replace(second=0, microsecond=0) - timedelta(minutes=time_window_after_minutes)
    end_time = start_time + 2 * timedelta(minutes=time_window_after_minutes)
    
    filtered = input_df.filter(
        (pl.col("Datum") >= start_time) & (pl.col("Datum") <= end_time)
    )

    def print_all_rows():
        for i in range(len(filtered)):
            print(filtered.row(i))

    if filtered.height > 0:
       #print_all_rows()
       return filtered.sort("Datum").tail(n)
    else:
        return None

row_n = 4
test = get_first_inputs_before_time(5, output_df.row(row_n), input_df)
print("Target row datetime:", dict(zip(output_df.columns, output_df.row(row_n)))["Datum"])
print("\nLast row before target time:", dict(zip(input_df.columns, test))["Datum"])

test_inputs = test.to_pandas()
test_output = output_df.row(row_n)
print("\nTest inputs:")
print(test_inputs)
print("\nTest output:")
print(test_output)


Target row datetime: 2022-04-26 20:00:00

Last row before target time: shape: (5,)
Series: 'Datum' [datetime[μs]]
[
	2022-04-26 19:42:00
	2022-04-26 19:48:00
	2022-04-26 19:53:00
	2022-04-26 20:12:00
	2022-04-26 20:43:00
]

Test inputs:
   LegacyId               Datum                  Operater    A1  \
0   1318759 2022-04-26 19:42:00     DARS Luka Loboda, PIC  NULL   
1   1318760 2022-04-26 19:48:00     DARS Luka Loboda, PIC  NULL   
2   1318761 2022-04-26 19:53:00     DARS Luka Loboda, PIC  NULL   
3   1318762 2022-04-26 20:12:00   DARS Janko Poženel, GNC  NULL   
4   1318763 2022-04-26 20:43:00   DARS Janko Poženel, GNC  NULL   

                                                  B1    C1    A2    B2  \
0  <p><strong>Nesreče</strong></p><p>Na ljubljans...  NULL  None  None   
1  <p><strong>Nesreče</strong></p><p>Na ljubljans...  NULL  None  None   
2  <p><strong>Nesreče</strong></p><p>Na ljubljans...  NULL  None  None   
3  <p><strong>Opozorila</strong></p><p>V sredo, 2...  NULL  None

In [33]:
from html.parser import HTMLParser
text_example = """<p><strong>Vreme</strong></p><p>Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!</p><p><strong>Omejitve za tovorna vozila</strong></p><p>Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton:<br>- v soboto, 1. 1., od 8. do 22. ure;<br>- v nedeljo, 2. 1., od 8. do 22. ure.</p><p>Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. <a href="https://www.promet.si/sl/splosne-omejitve" target="_blank">Več.</a></p><p><strong>Dela</strong></p><p>Na primorski avtocesti je ponovno odprt priključek <strong>Črni</strong><strong> Kal </strong>v obe smeri.</p>"""

class HTMLTextExtractor(HTMLParser):
    def __init__(self):
        super().__init__()
        self.text_parts = []

    def handle_data(self, data):
        self.text_parts.append(data)

    def handle_starttag(self, tag, attrs):
        if tag in {"p", "br", "div", "li"}:
            self.text_parts.append(" ")

    def handle_endtag(self, tag):
        if tag in {"p", "br", "div", "li"}:
            self.text_parts.append(" ")

    def get_text(self):
        # Join and normalize whitespace
        return ' '.join(''.join(self.text_parts).split())

def strip_html(html):
    parser = HTMLTextExtractor()
    parser.feed(html)
    return parser.get_text()

# Example usage
cleaned_text = strip_html(text_example)
print("\nCleaned text:")


Cleaned text:


In [ ]:
from bs4 import BeautifulSoup
from collections import Counter

input_df = input_df
# html is in column "B1"
html = """
<p><strong>Vreme</strong></p><p>Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!</p><p><strong>Omejitve za tovorna vozila</strong></p><p>Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton:<br>- v soboto, 1. 1., od 8. do 22. ure;<br>- v nedeljo, 2. 1., od 8. do 22. ure.</p><p>Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. <a href="https://www.promet.si/sl/splosne-omejitve" target="_blank">Več.</a></p><p><strong>Dela</strong></p><p>Na primorski avtocesti je ponovno odprt priključek <strong>Črni</strong><strong> Kal </strong>v obe smeri.</p>
"""

# Counter to collect tag frequencies
tag_counter = Counter()

# Loop through each HTML string
for html in input_df["B1"]:
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup.find_all(True):
        tag_counter[tag.name] += 1  # Count each tag once per appearance

# Print result
print("Tag counts:")
for tag, count in tag_counter.items():
    print(f"<{tag}>: {count}")


Tag counts:
<p>: 1983381
<strong>: 1533299
<br>: 611357
<a>: 217990
<u>: 16373
<em>: 326
<ul>: 69
<li>: 84


In [37]:
# Count contents of <strong> tags
strong_text_counter = Counter()

for html in input_df["B1"]:
    soup = BeautifulSoup(html, "html.parser")
    for tag in soup.find_all("strong"):
        text = tag.get_text(strip=True)
        if text:
            strong_text_counter[text] += 1

# Use most_common() to sort by count descending
sorted_counts = strong_text_counter.most_common()

# Print sorted results
print("Counts of <strong> tag contents (sorted):")
for text, count in sorted_counts:
    print(f"{text}: {count}")

Counts of <strong> tag contents (sorted):
Delo na cesti: 144823
Zastoji: 107741
Ovire: 101030
Nesreče: 74228
Vreme: 53525
Prireditve: 35301
Mejni prehodi: 32276
Opozorila: 24664
avtocesti: 15532
Tovorna vozila: 15450
Tovorni promet: 14021
Na: 13926
Delovne zapore: 11840
-: 11418
Zimske razmere: 10488
Dela: 9990
.: 9868
primorski avtocesti: 9043
Na gorenjski avtocesti: 8539
Popolne zapore:: 8352
,: 7732
:: 7660
Litija - Zagorje: 7133
Vršič: 6851
gorenjski avtocesti: 6820
primorski: 6816
štajerski avtocesti: 6024
Več o zaporah na zemljevidu.: 4874
Na štajerski avtocesti: 4839
predor Karavanke: 4773
zaprt: 4181
je: 4181
skozi predor Karavanke: 3955
izvoz Vrhnika: 3889
Na primorski avtocesti: 3637
v: 3531
Ljubljana: 3420
gorenjski: 3183
štajerski: 3103
Burja: 3041
Ostale ceste:: 3015
Ujma 2023: 2998
Ljubljana, Šentvid - Vodice bo: 2630
Tacnu na Cesti vstaje: 2630
zapore:: 2592
Popolne zapore nekaterih cest zaradi poškodovanih vozišč: 2537
zaprta: 2497
na: 2451
Na dolenjski avtocesti: 2444


In [ ]:

html = """
<p><strong>Vreme</strong></p><p>Ponekod po Sloveniji megla v pasovih zmanjšuje vidljivost. Prilagodite hitrost!</p><p><strong>Omejitve za tovorna vozila</strong></p><p>Po Sloveniji velja med prazniki omejitev za tovorna vozila z največjo dovoljeno maso nad 7,5 ton:<br>- v soboto, 1. 1., od 8. do 22. ure;<br>- v nedeljo, 2. 1., od 8. do 22. ure.</p><p>Od 30. decembra je v veljavi sprememba omejitve za tovorna vozila nad 7,5 ton. <a href="https://www.promet.si/sl/splosne-omejitve" target="_blank">Več.</a></p><p><strong>Dela</strong></p><p>Na primorski avtocesti je ponovno odprt priključek <strong>Črni</strong><strong> Kal </strong>v obe smeri.</p>
"""

json_format = {
    
    "Delo na cesti:": "",
    "Zastoji:": "",
    "Ovire:": "",
    "Nesreče:": "",
    "Vreme:": "",
    "Prireditve:": "",
    "Opozorila:": "",
    "Mejni prehodi:": "",
    "Delovne zapore:": "",
    "Zimske razmere:":"",
    "Dela:":"",
    "Popolne zapore::":"",
    "Popolne zapore:":"",
    "leftover_html": ""
}


In [105]:
from bs4 import BeautifulSoup
import re
import json
from IPython.display import display, HTML

# Template
json_format = {
    "Delo na cesti": "",
    "Zastoji": "",
    "Ovire": "",
    "Nesreče": "",
    "Vreme": "",
    "Prireditve": "",
    "Opozorila": "",
    "Mejni prehodi": "",
    "Delovne zapore": "",
    "Zimske razmere": "",
    "Dela": "",
    "Popolne zapore": "",
    "Omejitve za tovorna vozila": "",
    "leftover_html": ""
}

def parse_html_into_json(html, json_format, timestamp=None):
    # Normalize keys for matching
    normalized_keys = {
        re.sub(r":+", "", k).strip().lower(): k
        for k in json_format if k != "leftover_html"
    }

    soup = BeautifulSoup(html, "html.parser")
    elements = soup.find_all(["p", "br", "a"])

    current_heading = None
    content_map = {k: "" for k in json_format if k != "leftover_html"}
    leftover = []

    for tag in elements:
        text = tag.get_text(separator=" ", strip=True)

        # Skip empty elements
        if not text:
            continue

        # Check if tag starts with a known section heading
        for normalized_heading, real_key in normalized_keys.items():
            if text.lower().startswith(normalized_heading):
                current_heading = real_key
                # Remove the heading from the text itself
                cleaned = re.sub(f"^{re.escape(normalized_heading)}:?\s*", "", text, flags=re.I)
                if cleaned:
                    content_map[real_key] += cleaned + " "
                break
        else:
            if current_heading:
                content_map[current_heading] += text + " "
            else:
                leftover.append(text)

    # Strip trailing whitespace
    for key in content_map:
        content_map[key] = content_map[key].strip()

    json_format.update(content_map)
    json_format["leftover_html"] = "\n".join(leftover).strip()
    if timestamp:
        json_format["timestamp"] = timestamp.strftime("%d-%m-%Y %H:%M:%S")
    return json_format

    

# Call the function to parse the HTML and fill the json_format
row = input_df.select(["B1", "Datum"]).sample(1).row(0)
random_html, timestamp = row

print("Random HTML content from DataFrame:")
display(HTML(random_html))

myjson=parse_html_into_json(random_html, json_format, timestamp=timestamp)
print(json.dumps(myjson, indent=2, ensure_ascii=False))


Random HTML content from DataFrame:


<>:50: SyntaxWarning: invalid escape sequence '\s'
<>:50: SyntaxWarning: invalid escape sequence '\s'
C:\Users\turkf\AppData\Local\Temp\ipykernel_26828\964431768.py:50: SyntaxWarning: invalid escape sequence '\s'
  cleaned = re.sub(f"^{re.escape(normalized_heading)}:?\s*", "", text, flags=re.I)


{
  "Delo na cesti": "Več o delovnih zaporah v prometni napovedi . Več o delovnih zaporah v prometni napovedi Na ljubljanski južni obvoznici je zaradi rekonstrukcije zaprt priključek Ljubljana zahod, Vič v obe smeri do 25. 8. 2023. Predvidoma do četrtka, 31. 8., bo na ljubljanski severni obvoznici zaprt uvoz z Leskoškove ceste proti krožišču Tomačevo.",
  "Zastoji": "Na gorenjski avtocesti med priključkom Jesenice vzhod in predorom Karavanke proti Avstriji, 5 km. Občasno zapirajo predor. Voznikom, namenjenim proti Jesenicam in Kranjski Gori, priporočamo že izvoz Lesce. Na cesti Lesce - Bled.",
  "Ovire": "",
  "Nesreče": "Na primorski avtocesti za Brezovico proti Kopru oviran promet na odstavnem pasu. Na ljubljanski vzhodni obvoznici med Zaloško in razcepom Zadobrova proti Štajerski zaprt prehitevalni pas. Nastal je zastoj do Kozarij.",
  "Vreme": "Zaradi posledic ujme je omejen dostop nepooblaščenim osebam na nekatera območja. Na ta območja se ne odpravljajte, če niste dogovorjeni s p

In [66]:

columns = ["Datum", "A1", "B1", "C1","ContentPomembnoSLO", "ContentNesreceSLO" , "ContentNesreceSLO", "ContentVremeSLO", "ContentOvireSLO ", "ContentDeloNaCestiSLO", "ContentOpozorilaSLO"]
semantic_compare = {
  "Delo na cesti": "ContentDeloNaCestiSLO",
  "Zastoji": "",
  "Ovire": "ContentOvireSLO",
  "Nesreče": "ContentNesreceSLO",
  "Vreme": "ContentVremeSLO",
  "Prireditve": "",
  "Opozorila": "ContentOpozorilaSLO",
  "Mejni prehodi": "",
  "Delovne zapore": "",
  "Zimske razmere": "",
  "Dela": "",
  "Popolne zapore": "",
  "Omejitve za tovorna vozila": "",
  "leftover_html": "ContentPomembnoSLO"
}
# Call the function to parse the HTML and fill the json_format
#random_html = input_df["B1"].sample(1).item()  # Randomly sample one HTML entry from the DataFrame
#print("Random HTML content from DataFrame:")
#display(HTML(random_html))

#myjson=parse_html_into_json(random_html, json_format)
#print(json.dumps(myjson, indent=2, ensure_ascii=False))
    

In [72]:
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a.strip(), b.strip()).ratio()

# Sample the row (you already sampled the HTML)
row = input_df.sample(1)
random_html = row["B1"].item()
parsed = parse_html_into_json(random_html, json_format.copy())  # use copy to avoid overwriting

print("=== Semantic Comparison ===")
for key, column in semantic_compare.items():
    if not column:
        continue  # skip if no mapped column

    parsed_value = (parsed.get(key) or "").strip()
    actual_value = (row[column].item() if column in row else "") or ""
    actual_value = actual_value.strip()

    match_ratio = similarity(parsed_value, actual_value)
    
    if match_ratio < 0.8:
        print(f"❗️ Low similarity for key: {key} (ratio: {match_ratio:.2f})")

    print(f"🧠 {key} → {column}")
    print(f"✅ Similarity: {match_ratio:.2f}")
    print(f"📤 Extracted:\n{parsed_value[:300]}{'...' if len(parsed_value) > 300 else ''}")
    print(f"📥 Actual:\n{actual_value[:300]}{'...' if len(actual_value) > 300 else ''}")
    print("-" * 80)


=== Semantic Comparison ===
❗️ Low similarity for key: Delo na cesti (ratio: 0.78)
🧠 Delo na cesti → ContentDeloNaCestiSLO
✅ Similarity: 0.78
📤 Extracted:
Več o delovnih zaporah v prometni napovedi . Več o delovnih zaporah v prometni napovedi Na južni ljubljanski obvoznici bo zaradi rekonstrukcije zaprt priključek Ljubljana Vič v obe smeri od srede,16. 8., do 25. 8. 2023. Od nedelje do četrtka, 31. 8., bo na severni ljubljanski obvoznici zaprt uvoz iz...
📥 Actual:
<p><a href="https://www.promet.si/sl/aktualna-napoved" target="_blank">Več o delovnih zaporah v prometni napovedi</a>.</p><p><strong>Na južni ljubljanski obvoznici </strong>bo zaradi rekonstrukcije zaprt priključek Ljubljana Vič v obe smeri od srede,16. 8., do 25. 8. 2023.</p><p>Od nedelje do četrtk...
--------------------------------------------------------------------------------
🧠 Ovire → ContentOvireSLO
✅ Similarity: 0.96
📤 Extracted:
Na gorenjski avtocesti je pred galerijo Moste proti Karavankam zaprt vozni pas, okvara v

We now have a json structure for the data. now we need to compare the json input and the gennerated output to see if they match.

```python

In [153]:
import warnings
warnings.filterwarnings("ignore")
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained sentence embedding model
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

def get_similarity_score(output_message, json_string):
    # Compute embeddings
    embeddings = model.encode([output_message, json_string], convert_to_tensor=True)

    # Compute cosine similarity
    similarity_score = util.cos_sim(embeddings[0], embeddings[1])
    
    return similarity_score.item()


def parse_output_messages(output_row, columns):
    row_dict = dict(zip(columns, output_row))
    message = ""
    for column in ["content_01", "content_02", "content_03", "content_04", "content_05"]:
        if row_dict.get(column) is not None:
            if column == "content_05":
                if '\x00' == row_dict[column]:
                    continue
            message += f"{row_dict[column]} "
    return message.strip()

from tqdm import tqdm

def compare_json_to_output(input_df, output_df, n_examples=1, n_samples=5):
    traffic_examples = pd.DataFrame(columns=["Output Message", "Input_Message", "Input JSON", "Datum", "Similarity Score"])
    print(f"Comparing {n_examples} examples from output_df with input_df...")

    for i in tqdm(range(n_examples), desc="Examples"):  # outer loop progress bar
        try:
            traffic_examples_specific = pd.DataFrame(columns=["Output Message", "Input_Message", "Input JSON", "Datum", "Similarity Score"])
            timestamps = []
            similarity_scores = []
            timestamp_scores_dict = {}
            output_df_row = output_df.sample(1).row(0)  # Randomly sample one row from output_df
            input_rows = get_first_inputs_before_time(n_samples, output_df_row, input_df)

            # Inner loop over input rows with tqdm progress bar
            for _, input_row in tqdm(input_rows.to_pandas().iterrows(), total=len(input_rows), desc=f"Samples for example {i+1}", leave=False):
                try:
                
                    timestamps.append(input_row["Datum"])
                    html = input_row["B1"]
                    if isinstance(html, pl.Series):  # if it's a Polars Series, get the first item
                        html = html[0]
                    html = str(html) 
                    myjson = parse_html_into_json(html, json_format)
                    output = parse_output_messages(output_df_row, output_df.columns)
                    
                    full_string = ""
                    for key, value in myjson.items():
                        if value != "":
                            full_string += f"{value} "
                            
                    similarity_score = get_similarity_score(output, full_string)
                    # print(f"Similarity score: {similarity_score:.4f}")
                    similarity_scores.append(similarity_score)
                    timestamp_scores_dict[input_row["Datum"]] = similarity_score
                    
                    new_row = {
                        "Output Message": output,
                        "Input_Message": full_string,
                        "Input JSON": json.dumps(myjson, indent=2, ensure_ascii=False),
                        "Datum": input_row["Datum"],
                        "Similarity Score": similarity_score
                    }

                    new_df = pd.DataFrame([new_row])
                    traffic_examples_specific = pd.concat([traffic_examples_specific, new_df], ignore_index=True)

                except Exception as e:
                    print(f"Error processing input row: {e}")
                    continue
            
            # print(f"Average similarity score for example {i+1}: {sum(similarity_scores) / len(similarity_scores):.4f}")
            # print(similarity_scores)
            
            output_timestamp = output_df_row[0]  # or 0 — depends on your schema
            
            # print("*" * 80)
            # print(f"Output timestamp: {output_timestamp}")

            converted_dict = {str(k): v for k, v in timestamp_scores_dict.items()}
            # print(json.dumps(converted_dict, indent=2, ensure_ascii=False))
            
            best_example = traffic_examples_specific.loc[traffic_examples_specific["Similarity Score"].idxmax()]
            # print(f"Best example for output row {i+1}:", best_example["Similarity Score"])
            traffic_examples = pd.concat([traffic_examples, best_example.to_frame().T], ignore_index=True)
            
        except Exception as e:
            print(f"Error calculating average similarity score for example {i+1}: {e}")
            continue
        
    return traffic_examples

traffic_examples = compare_json_to_output(input_df, output_df, n_examples=len(output_df), n_samples=15)
# print("\nTraffic Examples DataFrame:")
# print(traffic_examples.head())

traffic_examples.to_csv("trainingdataset3.csv", index=False, encoding="utf-8")
print("\nTraffic examples saved to 'trainingdataset3.csv'.")


Comparing 23617 examples from output_df with input_df...


Examples:   0%|          | 113/23617 [02:21<8:09:05,  1.25s/it]


KeyboardInterrupt: 

In [163]:
#optimizzed version
import pandas as pd
import polars as pl
import json
from tqdm import tqdm
import warnings
from sentence_transformers import SentenceTransformer, util

warnings.filterwarnings("ignore")

# Assume these functions are defined elsewhere in your script
# from your_utils import parse_html_into_json, get_first_inputs_before_time

# It's good practice to load the model once
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')


def parse_output_messages(output_row, columns):
    # This function is fine as is, but we'll call it more efficiently
    # Assuming output_row is a tuple or list from iter_rows()
    row_dict = dict(zip(columns, output_row))
    message_parts = []
    for i in range(1, 6): # A slightly cleaner way to loop through content_01 to content_05
        col_name = f"content_0{i}"
        content = row_dict.get(col_name)
        if content and content != '\x00':
            message_parts.append(str(content))
    return " ".join(message_parts)


def compare_json_to_output_optimized(input_df, output_df, n_samples=15):
    """
    Optimized function to find the best input match for each output message.
    """
    best_results_list = []
    output_columns = output_df.columns

    print(f"Comparing {len(output_df)} output examples with {n_samples} input samples each...")

    # Iterate directly over output_df rows. This is much cleaner than sampling in a loop.
    for output_row_tuple in tqdm(output_df.iter_rows(), total=len(output_df), desc="Processing Output Examples"):

        try:
            # --- 1. Process the output ONCE per outer loop ---
            output_message = parse_output_messages(output_row_tuple, output_columns)
            if not output_message: # Skip if the output message is empty
                continue
            
            # Encode the single output message
            output_embedding = model.encode(output_message, convert_to_tensor=True)

            # --- 2. Get candidate inputs ---
            input_rows_df = get_first_inputs_before_time(n_samples, output_row_tuple, input_df).to_pandas()
            if input_rows_df.empty:
                continue

            # --- 3. Prepare all input texts for BATCH processing ---
            input_texts_to_encode = []
            processed_inputs = [] # Keep track of the full parsed data

            for _, input_row in input_rows_df.iterrows():
                html = str(input_row.get("B1", ""))
                
                # Assume parse_html_into_json is defined and returns a dict
                myjson = parse_html_into_json(html, json_format) 
                
                full_string = " ".join(value for value in myjson.values() if value)
                input_texts_to_encode.append(full_string)
                processed_inputs.append({
                    "Input_Message": full_string,
                    "Input_JSON_str": json.dumps(myjson, indent=2, ensure_ascii=False),
                    "Datum": input_row["Datum"]
                })

            # --- 4. BATCH encode all input texts at once ---
            input_embeddings = model.encode(input_texts_to_encode, convert_to_tensor=True)
            
            # --- 5. Compute all similarities in a single, fast operation ---
            cosine_scores = util.cos_sim(output_embedding, input_embeddings)
            # print(f"Cosine scores: {cosine_scores}")
            
            # Find the index of the best score
            best_sample_idx = cosine_scores[0].argmax().item()

            # --- 6. Select the best result ---
            best_input_data = processed_inputs[best_sample_idx]
            best_score = cosine_scores[0][best_sample_idx].item()
            # print(f"Best score for output row {output_row_tuple[0]}: {best_score:.4f}")
            
            # Append the single best result for this output to our list
            best_results_list.append({
                "Output Message": output_message,
                "Input_Message": best_input_data["Input_Message"],
                "Input JSON": best_input_data["Input_JSON_str"],
                "Datum": best_input_data["Datum"],
                "Similarity Score": best_score
            })

        except Exception as e:
            # It's good to know which row caused an error
            output_timestamp = output_row_tuple[0] 
            print(f"Error processing output row for timestamp {output_timestamp}: {e}")
            continue

    # --- 7. Create the final DataFrame ONCE from the list of results ---
    if not best_results_list:
        print("No valid examples were processed.")
        return pd.DataFrame()
        
    final_df = pd.DataFrame(best_results_list)
    return final_df

# Assuming input_df and output_df are already loaded as Polars DataFrames
# And json_format is defined
#sample the outputdf  
output_df_sample = output_df.sample(n=100)  # Sample 15 rows for testing
traffic_examples = compare_json_to_output_optimized(input_df, output_df_sample, n_samples=20)
traffic_examples.to_csv("trainingdataset_optimized.csv", index=False, encoding="utf-8")
# print("\nOptimized traffic examples saved to 'trainingdataset_optimized.csv'.")

Comparing 100 output examples with 20 input samples each...


Processing Output Examples: 100%|██████████| 100/100 [00:27<00:00,  3.68it/s]


Now we hhave a system that checks the output of the model against the concated json structure. And we can comare which specific innput matches the output the most. We will match the output to the input by comparing the output to the input and finding the closest match. and recording the similarity score and json structure along with the input and output in the training dataframe. 

B1  conntains tags like  vreme nesreče ovire.... whic is also i separate columns. Now tis information is often duplicated and i need to join it. 

In [ ]:
print("\nTest inputs:")
print(test_inputs.iloc[0])

def parse_input_message(input_row, columns):
    row_dict = dict(zip(columns, input_row))
    message = ""
    for column in ["Datum", "A1", "B1", "C1","ContentPomembnoSLO" , "ContentNesreceSLO", "ContentVremeSLO", "ContentOvireSLO ", "ContentDeloNaCestiSLO", "ContentOpozorilaSLO"]:
        value = row_dict.get(column)
        if value is not None and value != "NULL":
            if isinstance(value, datetime):
                clean_value = value.strftime("%Y-%m-%d %H:%M:%S")
            elif isinstance(value, str):
                clean_value = strip_html(value)
                #new line 
            message += f"{column}: {clean_value} "
    #print("Input message:", message.strip())
    return message.strip()
parse_input_message(test_inputs.iloc[0], test_inputs.columns)



Test inputs:
LegacyId                                                                     1318757
Datum                                                            2022-04-26 19:33:00
Operater                                                       DARS Luka Loboda, PIC
A1                                                                              NULL
B1                                 <p><strong>Nesreče</strong></p><p>Na ljubljans...
C1                                                                              NULL
A2                                                                              None
B2                                                                              None
C2                                 <div><p>Buy vignette for Slovenia online</p><p...
TitlePomembnoSLO                                                                None
ContentPomembnoSLO                                                              None
TitleNesreceSLO                                    

'Datum: 2022-04-26 19:33:00 B1: Nesreče Na ljubljanski severni obvoznici zaradi prometne nesreče med krožiščem Tomačevo in Bežigradom promet proti Kosezam poteka samo po enem prometnem pasu. Ovire Na štajerski avtocesti je zaradi živali zaprt prehitevalni pas med Šentjakobom in Zadobrovo proti Ljubljani. Opozorila V sredo, 27. 4., bo med 8. in 22. uro, zaradi praznika veljala splošna omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7,5 t. ContentNesreceSLO: Na ljubljanski severni obvoznici zaradi prometne nesreče med krožiščem Tomačevo in Bežigradom promet proti Kosezam poteka samo po enem prometnem pasu. ContentNesreceSLO: Na ljubljanski severni obvoznici zaradi prometne nesreče med krožiščem Tomačevo in Bežigradom promet proti Kosezam poteka samo po enem prometnem pasu. ContentOpozorilaSLO: V sredo, 27. 4., bo med 8. in 22. uro, zaradi praznika veljala splošna omejitev prometa tovornih vozil, katerih največja dovoljena masa presega 7,5 t.'

In [17]:
print("\nTest output:")
print(test_output)
def parse_output_messages(output_row, columns):
    row_dict = dict(zip(columns, output_row))
    message = ""
    for column in ["content_01", "content_02", "content_03", "content_04", "content_05"]:
        if row_dict.get(column) is not None:
            if column == "content_05":
                if '\x00' == row_dict[column]:
                    continue
            message += f"{row_dict[column]} "
    return message.strip()


parse_output_messages(test_output, output_df.columns)


Test output:
(datetime.datetime(2022, 4, 26, 20, 0), 'TMP-102.rtf', 'April 2022', 'Na južni ljubljanski obvoznici je zaradi pokvarjenega vozila  zaprt izvoz Ljubljana - jug proti dolenjski avtocesti.', 'Zaradi del je na severni ljubljanski obvoznici zaprt uvoz Nove Jarše proti štajerski avtocesti.', 'Na cesti Rogatec - Dobovec poteka promet zaradi prometne nesreče izmenično enosmerno.', 'Na Obrežju je povečana promet osebnih vozil pri vstopu v državo, v Jelšanah pa pri izstopu iz nje.', '\x00')


'Na južni ljubljanski obvoznici je zaradi pokvarjenega vozila  zaprt izvoz Ljubljana - jug proti dolenjski avtocesti. Zaradi del je na severni ljubljanski obvoznici zaprt uvoz Nove Jarše proti štajerski avtocesti. Na cesti Rogatec - Dobovec poteka promet zaradi prometne nesreče izmenično enosmerno. Na Obrežju je povečana promet osebnih vozil pri vstopu v državo, v Jelšanah pa pri izstopu iz nje.'

Okay now we have a way to smeantically check the output of the model against the input json structure. But we still need a parser that will check if events match. I believe that semantic similarity is not enough inn this case. We need to check if the events match. also maybe just count the number of events that match and dont match. Create a parser thhat will be able to identify 80% of the events that match and 20% that dont match. I dont need to go into to much detail here, just create a parser that will be able to identify the events that match and the ones that dont match.

In [169]:
import re
from enum import Enum
from typing import List, Optional, Dict, Any
from datetime import datetime
from pydantic import BaseModel, Field

# --- 1. Simplified Pydantic Models for Parsing ---
# It's easier to parse into a simpler structure first.
# RoadSection is too complex for a simple parser, so we'll capture location as a string.

class EventType(str, Enum):
    # I've added a few more based on your input text and common patterns
    CONGESTION = "Zastoj"                   # Zastoj
    ROADWORK = "Delo na cesti"                       # Dela na cesti, delovna zapora
    ACCIDENT = "Nesreča"                       # Nesreča
    OBSTACLE = "Ovira"                       # Ovire, predmet na cesti
    WEATHER = "Vreme"                         # Vreme, burja, sneg, poledica
    TRUCK_RESTRICTION = "Omejitve za tovorna vozila"     # Omejitve za tovorna vozila        # Mejni prehodi
    UNKNOWN = "unknown"                         # Could not classify

class ParsedTrafficEvent(BaseModel):
    """
    A simplified, practical model representing a single parsed traffic event.
    """
    event_type: EventType = Field(..., description="The classified type of the event.")
    location_description: str = Field(..., description="A description of the road or area affected.")
    
    delay_minutes: Optional[int] = Field(None, description="Estimated delay in minutes.")
    detour_available: Optional[bool] = Field(None, description="Indicates if a detour is mentioned.")
    
    source_text: str = Field(..., description="The original text segment this event was parsed from.")
    parse_timestamp: datetime = Field(default_factory=datetime.now)

# --- 2. Keyword and Regex Definitions (The "Brain" of the Parser) ---

# Map keywords to an EventType. Order matters: more specific keywords should come first.
# We use word stems to catch variations (e.g., "dela", "delovnih").
EVENT_TYPE_KEYWORDS: Dict[EventType, List[str]] = {
    EventType.TRUCK_RESTRICTION: ["omejitev prometa tovornih", "tovorna vozila"],
    EventType.ACCIDENT: ["nesreč", "nesr"],
    EventType.ROADWORK: ["del na cesti", "delovna zapora", "delovnih zaporah", "montažni most"],
    EventType.CONGESTION: ["zastoj", "zamuda", "promet upočasnjen", "kolona", "zgostitev prometa"],
    EventType.OBSTACLE: ["ovira", "predmet na cesti", "razsut tovor", "žival"],
    EventType.WEATHER: ["vremensk", "burja", "sneg", "poledic", "megla", "vrem", "piha"],
}

def _find_event_type(text: str) -> EventType:
    """Finds the event type based on keywords in the text."""
    lower_text = text.lower()
    for event_type, keywords in EVENT_TYPE_KEYWORDS.items():
        if any(keyword in lower_text for keyword in keywords):
            return event_type
    return EventType.UNKNOWN

def _extract_delay(text: str) -> Optional[int]:
    """Extracts delay time in minutes. Averages ranges."""
    # Matches patterns like "zamuda 5 - 10 minut", "zamuda 10 minut"
    match = re.search(r"zamuda\s*(\d+)(?:\s*-\s*(\d+))?\s*minut", text.lower())
    if match:
        start_delay = int(match.group(1))
        end_delay = int(match.group(2)) if match.group(2) else start_delay
        return (start_delay + end_delay) // 2
    return None

def _extract_location(text: str) -> str:
    """A simple location extractor. It cleans up some common clutter."""
    # This can be improved, but for an 80/20 solution, we remove known non-location phrases.
    location = text.strip()
    clutter = [
        "Več o delovnih zaporah v prometni napovedi",
        ", zamuda 5 - 10 minut",
        ", zamuda 10 minut",
        "."
    ]
    for c in clutter:
        location = location.replace(c, "").strip()
    return location

# --- 3. The Main Parsing Function ---

def parse_traffic_text(text: str) -> List[ParsedTrafficEvent]:
    """
    Parses a block of unstructured traffic text into a list of structured event objects.

    Args:
        text (str): The raw input text containing multiple traffic events.

    Returns:
        List[ParsedTrafficEvent]: A list of parsed event objects.
    """
    events: List[ParsedTrafficEvent] = []
    
    # Split text into lines. This is a robust starting point for traffic reports.
    # We filter out empty lines.
    lines = [line.strip() for line in text.split('.') if line.strip()]

    for line in lines:
        event_type = _find_event_type(line)

        # We only create an event if it's something we can classify.
        # This ignores generic lines like "Več o..." if they don't contain other keywords.
        if event_type != EventType.UNKNOWN:
            delay = _extract_delay(line)
            detour = "obvoz" in line.lower()
            location = _extract_location(line)

            event = ParsedTrafficEvent(
                event_type=event_type,
                location_description=location,
                delay_minutes=delay,
                detour_available=detour if detour else None, # Only set if True
                source_text=line
            )
            events.append(event)
    
    return events


# --- 4. Example Usage ---
if __name__ == "__main__":
    
    df = pd.read_csv("trainingdataset_optimized.csv", encoding="utf-8")
    sample = df.sample(n=1)  # Sample one row for testing
    input_text = sample["Input_Message"].iloc[0] # Get the first input message for testing
    output_text = sample["Output Message"].iloc[0] # Get the first output message for testing

    print("--- Parsing Input Text ---")
    print(input_text)  # Display the input text
    parsed_events = parse_traffic_text(input_text)
    
    print("--- Parsing Output Text ---")
    print(output_text)  # Display the output text
    parsed_events2 = parse_traffic_text(output_text)  # Parse both input and output texts

    print(f"\nFound {len(parsed_events)}/{len(parsed_events2)} events")
    print("-" * 25)
    
    

    for i, event in enumerate(parsed_events):
        print(f"Event #{i+1}:")
        # .model_dump_json is a Pydantic v2 method, for v1 use .json()
        try:
            print(event.model_dump_json(indent=2, exclude={'parse_timestamp'}))
        except AttributeError:
            print(event.json(indent=2, exclude={'parse_timestamp'})) # For Pydantic v1
        print("-" * 25)
        
    for i, event in enumerate(parsed_events2):
        print(f"Event #{i+1}:")
        # .model_dump_json is a Pydantic v2 method, for v1 use .json()
        try:
            print(event.model_dump_json(indent=2, exclude={'parse_timestamp'}))
        except AttributeError:
            print(event.json(indent=2, exclude={'parse_timestamp'}))

--- Parsing Input Text ---
Na severni ljubljanski obvoznici je izvoz Nove Jarše iz smeri Zadobrove možen samo za voznike, ki pripeljejo iz smeri Domžal. Na pomurski avtocesti bo do 8. julija zaprt priključek Pernica proti Mariboru. Obvoz preko priključka Lenart. Na mariborski vzhodni obvoznici je zaradi popravila pokritega vkopa Malečnik promet urejen dvosmerno. Po enem pasu v vsako smer. Na italijanski strani prelaza Predel , med Predelom in Trbižem, pri predoru ob Rabeljskem jezeru, bodo popolne nočne zapore ceste do 30. 6. 2023, vsako noč med 20. in 6. uro zjutraj. Med 6. in 20. uro čez dan pa bo promet potekal izmenično enosmerno. Več o delovnih zaporah v prometni napovedi . Več o delovnih zaporah v prometni napovedi Promet je povečan na cestah proti večjim mestom in na mestnih vpadnicah zaradi jutranje prometne konice. Na pomurski avtocesti med priključkoma Pince in Lendava proti Mariboru, okvara tovornega vozila, zaprt vozni pas. Okvara vozila na štajerski avtocesti med počivališ